In [1]:
import numpy as np
import random
from typing import List, Tuple

### Create synthetic data

In [2]:
# Simple 1D function to estimate
def target_function(x: float) -> float:
    return 2 * x**2 + 3 * x + 1

# Generate synthetic data
X = np.linspace(-10, 10, 100)
Y = target_function(X) + np.random.normal(0, 1, 100)  # Add some noise


### Create class for Genetic Algorithm

In [3]:

class Individual:
    def __init__(self):
        self.genes = np.random.uniform(-1, 1, 3)  # [a, b, c] for ax^2 + bx + c
        self.fitness = 0

    def calculate_fitness(self):
        predictions = self.genes[0] * X**2 + self.genes[1] * X + self.genes[2]
        self.fitness = -np.mean((Y - predictions)**2)  # Negative MSE

class GeneticAlgorithm:
    def __init__(self, population_size: int = 50):
        self.population = [Individual() for _ in range(population_size)]

    def evolve(self, mutation_rate: float, crossover_rate: float):
        for ind in self.population:
            ind.calculate_fitness()
        
        self.population.sort(key=lambda x: x.fitness, reverse=True)
        new_population = self.population[:2]  # Elitism

        while len(new_population) < len(self.population):
            if random.random() < crossover_rate:
                parent1, parent2 = random.sample(self.population[:10], 2)
                child = Individual()
                child.genes = (parent1.genes + parent2.genes) / 2
                new_population.append(child)
            else:
                new_population.append(random.choice(self.population[:10]))

        for ind in new_population[2:]:
            if random.random() < mutation_rate:
                ind.genes += np.random.normal(0, 0.1, 3)

        self.population = new_population


### Implement K-Arm Bandit Algorithm

In [4]:
class Arm:
    def __init__(self, mutation_rate: float, crossover_rate: float):
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.value = 0
        self.pulls = 0

class KArmBandit:
    def __init__(self, arms: List[Arm]):
        self.arms = arms

    def select_arm(self, epsilon: float) -> Arm:
        if random.random() < epsilon:
            return random.choice(self.arms)
        else:
            return max(self.arms, key=lambda arm: arm.value)

    def update_arm(self, arm: Arm, reward: float):
        arm.pulls += 1
        arm.value += (reward - arm.value) / arm.pulls


In [5]:

def main():
    arms = [
        Arm(0.01, 0.7), Arm(0.05, 0.7), Arm(0.1, 0.7),
        Arm(0.01, 0.8), Arm(0.05, 0.8), Arm(0.1, 0.8),
        Arm(0.01, 0.9), Arm(0.05, 0.9), Arm(0.1, 0.9)
    ]
    
    bandit = KArmBandit(arms)
    ga = GeneticAlgorithm()
    epsilon = 0.3
    generations = 100

    for generation in range(generations):
        selected_arm = bandit.select_arm(epsilon)
        
        old_best_fitness = ga.population[0].fitness
        ga.evolve(selected_arm.mutation_rate, selected_arm.crossover_rate)
        new_best_fitness = ga.population[0].fitness
        
        reward = new_best_fitness - old_best_fitness
        bandit.update_arm(selected_arm, reward)
        
        epsilon *= 0.99  # Decay epsilon
        
        if generation % 10 == 0:
            best = ga.population[0]
            print(f"Generation {generation}: Best fitness = {best.fitness:.4f}")
            print(f"Estimated function: {best.genes[0]:.2f}x^2 + {best.genes[1]:.2f}x + {best.genes[2]:.2f}")
            print(f"Selected arm: MR = {selected_arm.mutation_rate}, CR = {selected_arm.crossover_rate}")

    best = ga.population[0]
    print("\nFinal estimation:")
    print(f"{best.genes[0]:.2f}x^2 + {best.genes[1]:.2f}x + {best.genes[2]:.2f}")
    print(f"True function: 2.00x^2 + 3.00x + 1.00")

    print("\nArm statistics:")
    for i, arm in enumerate(bandit.arms):
        print(f"Arm {i}: MR = {arm.mutation_rate}, CR = {arm.crossover_rate}, Value = {arm.value:.4f}, Pulls = {arm.pulls}")

if __name__ == "__main__":
    main()

Generation 0: Best fitness = -2584.3154
Estimated function: 0.96x^2 + 0.05x + 0.73
Selected arm: MR = 0.01, CR = 0.7
Generation 10: Best fitness = -895.5297
Estimated function: 1.46x^2 + 0.22x + 0.64
Selected arm: MR = 0.05, CR = 0.8
Generation 20: Best fitness = -490.1023
Estimated function: 1.66x^2 + 0.34x + 0.60
Selected arm: MR = 0.05, CR = 0.8
Generation 30: Best fitness = -207.3402
Estimated function: 2.01x^2 + 0.54x + 0.43
Selected arm: MR = 0.05, CR = 0.8
Generation 40: Best fitness = -160.9663
Estimated function: 2.06x^2 + 0.88x + 0.48
Selected arm: MR = 0.05, CR = 0.8
Generation 50: Best fitness = -104.5492
Estimated function: 2.05x^2 + 1.29x + 0.47
Selected arm: MR = 0.05, CR = 0.8
Generation 60: Best fitness = -54.8939
Estimated function: 2.05x^2 + 1.77x + 0.18
Selected arm: MR = 0.05, CR = 0.9
Generation 70: Best fitness = -41.5446
Estimated function: 2.04x^2 + 1.93x + 0.21
Selected arm: MR = 0.05, CR = 0.9
Generation 80: Best fitness = -26.1168
Estimated function: 2.05x^2